In [18]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback
from keras.layers import Dense,Reshape,Input,BatchNormalization,Dropout,concatenate,Flatten,Lambda,Resizing,CuDNNLSTM,concatenate,Layer,LSTM
from keras import  Model,layers
from tensorflow.keras.applications import VGG16 
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os

In [20]:
def find_removable(n,batch_size=64):
      return n%batch_size

In [21]:
base_path_oad = '/content/drive/MyDrive/Colab Notebooks/OAD/OAD/Vincent/Z/9/'
base_path_ntu = '/content/drive/MyDrive/Colab Notebooks/NTU/Vincent/ntu_3cam/'
input_shape= (25,40, 3)

### OAD
train_x_oad = np.load(base_path_oad+'train_x.npy')
test_x_oad = np.load(base_path_oad+'test_x.npy')
train_y_oad = np.load(base_path_oad+'train_y.npy')
test_y_oad = np.load(base_path_oad+'test_y.npy')
### NTU
train_x_ntu = np.load(base_path_ntu+'train_x.npy')
test_x_ntu = np.load(base_path_ntu+'test_x.npy')
train_y_ntu = np.load(base_path_ntu+'train_y.npy')
test_y_ntu = np.load(base_path_ntu+'test_y.npy')

print(train_x_oad.shape)
print(train_x_ntu.shape)

(9900, 25, 40, 3)
(71708, 25, 40, 3)


In [22]:
batch_size=64

n_train_y = []
n_test_y = []

pour_oad = [0,0,0,0,0,0,0]
pour_ntu = [0,0,0,0,0,0,0,0,0]
# train_x_ntu = 
for i in range(len(train_y_oad)):
  new = np.concatenate((train_y_oad[i],pour_oad))
  n_train_y.append(new)

for i in range(len(test_y_oad)):
  new = np.concatenate((test_y_oad[i],pour_oad))
  n_test_y.append(new)

train_y_oad = np.asarray(n_train_y)
test_y_oad = np.asarray(n_test_y)

n_train_y = []
n_test_y = []

for i in range(len(train_y_ntu)):
  new = np.concatenate((pour_ntu,train_y_ntu[i]))
  n_train_y.append(new)

for i in range(len(test_y_ntu)):
  new = np.concatenate((pour_ntu,test_y_ntu[i]))
  n_test_y.append(new)

train_y_ntu = np.asarray(n_train_y)
test_y_ntu = np.asarray(n_test_y)

n_train_y = []
n_test_y = []

train_x = np.concatenate((train_x_oad,train_x_ntu))
train_y = np.concatenate((train_y_oad,train_y_ntu))
test_x = np.concatenate((test_x_oad,test_x_ntu))
test_y = np.concatenate((test_y_oad,test_y_ntu))

for i in range(41, 46):
    rand_state = np.random.RandomState(i)
    rand_state.shuffle(train_x)
    rand_state.seed(i)
    rand_state.shuffle(train_y)

for i in range(11, 17):
    rand_state = np.random.RandomState(i)
    rand_state.shuffle(test_x)
    rand_state.seed(i)
    rand_state.shuffle(test_y)

print(train_x.shape)
print(train_y.shape)

tab_train = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# tab_test = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

n_train_x = []
n_train_y = []
# n_test_x = []
# n_test_y = []

for i in range(len(train_y)):
  if tab_train[np.argmax(train_y[i], axis=-1)] < 1100:
    n_train_x.append(train_x[i])
    n_train_y.append(train_y[i])
    tab_train[np.argmax(train_y[i], axis=-1)] += 1

# for i in range(len(test_y)):
#   if tab_test[np.argmax(test_y[i], axis=-1)] < 1100:
#     n_test_x.append(test_x[i])
#     n_test_y.append(test_y[i])
#     tab_test[np.argmax(test_y[i], axis=-1)] += 1

train_x = np.asarray(n_train_x)
train_y = np.asarray(n_train_y)

# test_x = np.asarray(n_test_x)
# test_y = np.asarray(n_test_y)

n = find_removable(len(train_x),batch_size)
if n!=0:
  train_x = train_x[:-n]
  train_y = train_y[:-n]

print('lost',n)

print(tab_train)
# print(tab_test)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
train = (train_x,train_y)
test = (test_x,test_y)
# val = (val_x,val_y)

(81608, 25, 40, 3)
(81608, 16)
lost 0
[1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100, 1100]
(17600, 25, 40, 3)
(17600, 16)
(69789, 25, 40, 3)
(69789, 16)


In [23]:
%tensorflow_version 2.x
print("Tensorflow version " + tf.__version__)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

Tensorflow version 2.8.0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.47.219.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.47.219.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [32]:
with strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  inp=Input(np.asarray(input_shape))
  # output = inp
  modelOAD = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/NTU/meilleurs/95_oad_z_9.h5')
  modelNTU = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/NTU/meilleurs/90_5_ntu.h5')
  modelOAD.trainable = False
  modelNTU.trainable = False
  modelo = modelOAD(inp)
  modeln = modelNTU(inp)
  output = concatenate([modelo,modeln])
  output = Dense(16)(output)
  output = Dense(16)(output)
  output = Dense(16, activation='softmax',kernel_regularizer='l1')(output)
  model = Model(inputs=inp,
                outputs=output)
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer='adam',
                metrics=['accuracy'])
  model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 25, 40, 3)]  0           []                               
                                                                                                  
 model_8 (Functional)           (None, 9)            14984009    ['input_11[0][0]']               
                                                                                                  
 model_11 (Functional)          (None, 7)            23138119    ['input_11[0][0]']               
                                                                                                  
 concatenate_9 (Concatenate)    (None, 16)           0           ['model_8[0][0]',                
                                                                  'model_11[0][0]']         

In [33]:
history=model.fit(train_x, train_y,
              epochs=200,
              batch_size=batch_size,
              verbose=1)
keras.models.save_model(model, base_path_oad+'ntuoad_z.h5')
print(model.evaluate(test[0],test[1]))

Epoch 1/200
275/275 [==============================] - 12s 18ms/step - loss: 2.5960 - accuracy: 0.6293
Epoch 2/200
275/275 [==============================] - 5s 18ms/step - loss: 1.2978 - accuracy: 0.8814
Epoch 3/200
275/275 [==============================] - 5s 18ms/step - loss: 1.0285 - accuracy: 0.9116
Epoch 4/200
275/275 [==============================] - 5s 18ms/step - loss: 0.9208 - accuracy: 0.9188
Epoch 5/200
275/275 [==============================] - 5s 18ms/step - loss: 0.8534 - accuracy: 0.9214
Epoch 6/200
275/275 [==============================] - 5s 19ms/step - loss: 0.8052 - accuracy: 0.9238
Epoch 7/200
275/275 [==============================] - 5s 18ms/step - loss: 0.7672 - accuracy: 0.9257
Epoch 8/200
275/275 [==============================] - 5s 18ms/step - loss: 0.7366 - accuracy: 0.9280
Epoch 9/200
275/275 [==============================] - 5s 18ms/step - loss: 0.7117 - accuracy: 0.9290
Epoch 10/200
275/275 [==============================] - 5s 18ms/step - loss: 0.69